In [13]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics
from matplotlib import pyplot as plt

df = pd.read_csv("bikebuyers_original.csv")
df

,ID,Marital Status,Gender,Income,Children,Cars,Age,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
0,19147,Married,Male,40000,0,1,42,Bachelors,Professional,No,0-1 Miles,North America,No
1,22296,Married,Male,70000,0,1,38,Bachelors,Professional,No,0-1 Miles,North America,No
2,25983,Married,Male,70000,0,1,43,Bachelors,Professional,No,0-1 Miles,North America,No
3,29134,Married,Male,60000,4,3,42,Bachelors,Skilled Manual,No,10+ Miles,North America,No
4,27388,Married,Male,60000,3,2,66,Bachelors,Management,No,1-2 Miles,North America,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,14798,Single,Female,10000,4,2,41,Partial High School,Manual,Yes,0-1 Miles,Europe,Yes
996,16259,Single,Female,10000,4,2,40,Partial High School,Manual,Yes,0-1 Miles,Europe,Yes
997,16410,Single,Female,10000,4,2,41,Partial High School,Manual,Yes,0-1 Miles,Europe,Yes
998,28957,Single,Female,120000,0,4,34,Partial High School,Professional,Yes,10+ Miles,Pacific,Yes


In [14]:
from sklearn import preprocessing

y = df["Purchased Bike"]
X = df.drop(columns=['Purchased Bike'])
X = pd.get_dummies(X, drop_first=True)
X = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(X), columns=X.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
X.head()

,ID,Income,Children,Cars,Age,Marital Status_Single,Gender_Male,Education_Graduate Degree,Education_High School,Education_Partial College,...,Occupation_Manual,Occupation_Professional,Occupation_Skilled Manual,Home Owner_Yes,Commute Distance_1-2 Miles,Commute Distance_10+ Miles,Commute Distance_2-5 Miles,Commute Distance_5-10 Miles,Region_North America,Region_Pacific
0,0.441644,0.1875,0.0,0.25,0.265625,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.612349,0.3750,0.0,0.25,0.203125,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.812219,0.3750,0.0,0.25,0.281250,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.983032,0.3125,0.8,0.75,0.265625,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.888383,0.3125,0.6,0.50,0.640625,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [15]:
# XGBoost needs the label in the format of [0, 1, 2, 3, ..., n - 1]. It can't handle categorical values
from sklearn.preprocessing import LabelEncoder
y_encoded = LabelEncoder().fit(y).transform(y) # This line converts the y from ['yes', 'no', 'no'] to [1, 0, 0], much like dummy coding
# Now we have to resplit the data with these encoded y values:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=1)

In [16]:
from sklearn.linear_model import LogisticRegression
import sklearn.linear_model as lm
from sklearn import metrics

model_logistic = lm.LogisticRegression(max_iter=1000).fit(X_train, y_train)
pd.DataFrame({'Coefficients on smoking':model_logistic.coef_[0]}, index=X_train.columns)  # code in the book did not work!!

,Coefficients on smoking
ID,0.637910
Income,1.473912
Children,-0.591760
Cars,-1.609268
Age,0.198640
Marital Status_Single,0.535380
Gender_Male,-0.134414
Education_Graduate Degree,-0.188730
Education_High School,0.162736
Education_Partial College,-0.226290
